# 데이터 분석하는 폴더입니다

In [1]:
import pandas as pd
# 형태소분석 라이브러리
from konlpy.tag import Okt
# 자연어 처리 라이브러리
import nltk
import os

In [2]:
# 형태소 분석기 객체 생성
okt = Okt()

In [ ]:
targetPath = ''
filename = 'duple_reivews.csv'

In [4]:
fullPath = targetPath + filename
df = pd.read_csv(fullPath, header=None, names=['text'])
df['text'] = df['text'].astype(str)

In [ ]:
train_data = []

In [6]:
stopwords = [
    # --- 조사 (Josa) 및 어미 (Eomi) ---
    '이', '가', '은', '는', '을', '를', '의', '에', '에서', '에게', '께', '로', '으로', 
    '와', '과', '보다', '처럼', '만큼', '같이', '까지', '마저', '조차', '부터', 
    '이나', '나', '이며', '며', '등', '하다', '한다', '하고', '하니', '하면', 
    '되어', '되다', '되고', '되니', '입니다', '습니다', 'ㅂ니다', '어요', '아요', '다', 
    '고', '면', '며', '게', '지', '죠',

    # --- 접속 부사 (Conjunctive Adverbs) ---
    '그리고', '그러나', '하지만', '그런데', '그래서', '그러면', '그러므로', '따라서', 
    '또한', '또는', '및', '즉', '한편', '반면에', '근데'

    # --- 대명사 (Pronouns) ---
    '나', '저', '우리', '저희', '너', '너희', '당신', '그', '그녀', '그들', '누구', 
    '무엇', '어디', '언제', '어느', '이것', '그것', '저것', '여기', '거기', '저기', 
    '이쪽', '그쪽', '저쪽',

    # --- 수사 (Numerals) ---
    '하나', '둘', '셋', '넷', '다섯', '여섯', '일곱', '여덟', '아홉', '열',
    '일', '이', '삼', '사', '오', '육', '칠', '팔', '구', '십', '백', '천', '만',
    '첫째', '둘째', '셋째',

    # --- 일반 부사 및 명사 (General Adverbs and Nouns) ---
    '바로', '때', '것', '수', '일', '문제', '경우', '부분', 
    '내용', '결과', '자체', '가지',

    # --- 의존 명사 (Dependent Nouns) ---
    '뿐', '대로', '만큼', '만', '지', '따름', '나름', '김에', '터',

    # --- 감탄사 및 기타 (Interjections and Others) ---
    '아', '아이고', '아이구', '아하', '어', '그래', '응', '네', '예', '아니',
]

In [7]:
for text in df['text']:
    tokens = okt.morphs(text)
    filtered = [w for w in tokens if w not in stopwords]
    train_data.append(filtered)

In [ ]:
all_tokens = [word for tokens in train_data for word in tokens]
vocabulary = list(set(all_tokens))
print("모든 토큰 개수:", len(all_tokens))
print("단어 사전의 단어 개수:", len(vocabulary))

모든 토큰 개수: 517457
단어 사전의 단어 개수: 29622


In [ ]:
def create_feature_set(tokens, vocabulary) :
    token_set = set(tokens)
    features = { word : (word in token_set) for word in vocabulary }
    return features

In [13]:
train_feature_datas = [
    create_feature_set(tokens,vocabulary)
    for tokens in train_data 
]

In [14]:
print(train_feature_datas[0])

{'기분': False, '메니': False, '분이시면': False, '되어있구요': False, '한잔': False, '작았으나': False, '뽀송뽀송합니다': False, '좋겟다': False, '참을만은': False, '물세': False, '들어가자말자': False, '묵으려': False, '많았지만': False, '컨잘나왓': False, 'ㅠㅡㅜ': False, '보냈더니': False, '찾아보기가': False, '묻어있었음': False, '기물': False, '인조': False, '약한게': False, '였어': False, '자나': False, '따지듯이': False, '일찍': False, '따뜻해지지': False, '빠르더라구요': False, '오성': False, '가습기': False, '끄지도': False, '아따': False, '않있는데도': False, '뜬한': False, '더오래': False, '구관': False, '뱉어놓았고': False, '찾아보던': False, '뭐라하지': False, '역사': False, '하고싶어여': False, '금암동': False, '발렛파킹': False, '했습니다': False, '알려주셨으면': False, '오래되어서': False, '전선': False, '해주거나': False, '잠들수있을': False, '늦어진': False, '들어오긴': False, '깨끗햏': False, '있었는데도': False, '10월': False, '잡을수있어서': False, '쾌적해도': False, '답답하네요': False, '이벤트': False, '닦다': False, '셍': False, '까지라서여유롭게놀수있어서너무좋아요가격대비': False, '작긴하지만': False, '였던걸': False, '틀어줘야': False, 'ㅠㅠㅠㅠㅠㅠ': False, '놀랬어요': False, '특실': False, '올라오는거': False, 

In [ ]:
print("모델 학습 시작")
model = nltk.NaiveBayesClassifier.train(train_feature_datas)
print("모델 학습 완료")